In [52]:
import pandas as pd
import xgboost as xgb
from sklearn.cross_validation import train_test_split,StratifiedKFold
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn import svm 
from sklearn.naive_bayes import BernoulliNB,GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import numpy as np

### GBDT模型

In [56]:
def model_xgb(train,test):
    #数据准备
    dtrain = xgb.DMatrix(train.iloc[:, :-1] , label=train.iloc[:, -1] )
    dval = xgb.DMatrix(test.iloc[:, :-1], label=test.iloc[:, -1])

    #log
    params = {
        'booster': 'gbtree',
        'objective': 'binary:logistic',
        'scale_pos_weight': 1100 / 200, #类别权重
        'eval_metric': 'auc',
        'gamma': 0.01,
        'silent': 1,
        'max_depth': 5,
        'lambda': 0.1,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'min_child_weight': 0.1,
        'eta': 0.016,
        'nthread': 5
    }

    evals_result = {}
    watchlist = [(dtrain,'train'),(dval,'val')]
    xgb_model = xgb.train(params,dtrain,num_boost_round=1000,early_stopping_rounds=200,verbose_eval=False,
                          evals=watchlist,evals_result=evals_result)
     
    result = max(evals_result['val']['auc'])
    return xgb_model,result
model_xgb

<function __main__.model_xgb>

In [33]:
# model_xgb,auc = model_xgb(train)
# y_pro = model_xgb.predict(test_ma)
# # df = pro_optimization(y_pro,test.iloc[:,-1])
# ks_plot(test.iloc[:,-1],y_pro,'GBDT')
# #特征权重
# dic = model.get_fscore()
# fea_rank = pd.DataFrame.from_dict([dic])
# fea_rank.T.sort_values(by=0,ascending= False).to_csv('./data/features_rank_model_nor.csv')
# fea_rank.T.sort_values(by=0,ascending= False)

### LR模型

In [34]:
#class_weight = {1:0.9,0:0.1}
def model_lr(train_X,train_Y):
    lg = LogisticRegressionCV(class_weight='balanced')
    lg.fit(train_X,train_Y)  
    return lg
lg = LogisticRegressionCV(class_weight='balanced')
lg

LogisticRegressionCV(Cs=10, class_weight='balanced', cv=None, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0)

In [35]:
# y_pre = lg.predict(val_X)
# print 'accuracy:',accuracy_score(val_Y,y_pre)
# print 'precision:',precision_score(val_Y,y_pre)
# print 'recall:',recall_score(val_Y,y_pre)
# print 'f1_score:',f1_score(val_Y,y_pre)

# y_pro_lr = lg.predict_proba(val_X)
# y_pro_lr = [x[1] for x in y_pro_lr]
# ks_plot(val_Y,y_pro_lr,'LR')

### SVM 模型

In [36]:
def model_svm(train_X,train_Y):
    m_svm = svm.SVC(class_weight='balanced',probability=True)
    m_svm.fit(train_X,train_Y)
    return m_svm
svm.SVC(class_weight='balanced',probability=True)

SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [37]:
# y_pro_svm = model_svm.predict_proba(val_X)
# y_pro_svm = [x[1] for x in y_pro_svm]
# ks_plot(val_Y,y_pro_svm,'SVM')
# svm.SVC(class_weight='balanced',probability=True)

### NB模型

In [38]:
def model_NB(train_X,train_Y):
    nb = GaussianNB()
    nb.fit(train_X,train_Y)
    return nb
GaussianNB()


GaussianNB(priors=None)

### KNN模型

In [39]:
def model_KNN(train_X,train_Y):
    knn = KNeighborsClassifier(n_neighbors=10)
    knn.fit(train_X,train_Y)
    return knn
KNeighborsClassifier(n_neighbors=10)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform')

### Neural Network

In [40]:
def model_NN(train_X,train_Y):
    nn = MLPClassifier(activation='logistic')
    nn.fit(train_X,train_Y)
    return nn
MLPClassifier(activation='logistic')

MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

### DT模型

In [41]:
def model_DT(train_X,train_Y):
    dt = DecisionTreeClassifier(class_weight='balanced',max_depth='8')
    dt.fit(train_X,train_Y)
    return dt
DecisionTreeClassifier()

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

### 加载数据
- 5折交叉验证

In [42]:
#订单样本
# samples = pd.read_csv('./new_out/samples_all_fillna_nor.csv')  #所有客户
# samples = pd.read_csv('./data/samples_all_fillna.csv',index_col=0)
# samples = pd.read_csv('./data/samples_pca.csv',index_col=0)
# samples = pd.read_csv('./new_out/old_custom_samples_all_fillna_nor.csv',index_col=0)  #老客户

# cols_filter =['实收资本','注册时间','流动比率','资产负债率','存货周转天数','净资产','主营业务利润率','应收账款周转天数',
#               '下游客户情况','企业征信','涉诉信息','有无贷款','提前还款占比','逾期占比','平均逾期天数','5-10天占比','4天以内占比',
#               '账期订单交易量均值','交易量方差','交易量最小值','交易额最小值','交易量均值','采购频率（订单总数/合作月数）','供应商数量',
#               '交易额均值','账期订单交易额均值','交易额方差','交易稳定性','交易额最大值','合作时间（月）',
#               '账期订单占比','交易额总量','交易总量','交易量最大值','11天以上占比','最近连续交易月数','账期合作时间（月）',
#               '采购聚集程度（订单总数/供应商数）','订单总数','label']
# samples = samples[cols_filter]
# samples.shape,samples.columns

In [43]:
#不合并企业征信
# samples = pd.read_csv('./data/samples_all_fillna_nor.csv',index_col=0)
# cols_filter =['实收资本','注册时间','流动比率','资产负债率','存货周转天数','净资产','主营业务利润率','应收账款周转天数',
#               '下游客户情况','征信不良', '征信关注', '征信欠息','征信正常','涉诉信息','有无贷款','提前还款占比','逾期占比',
#               '5-10天占比','4天以内占比',
#               '账期订单交易量均值','交易量方差','交易量最小值','交易额最小值','交易量均值','采购频率（订单总数/合作月数）','供应商数量',
#               '交易额均值','账期订单交易额均值','交易额方差','交易稳定性（近三个月交易量变化均值）','交易额最大值','合作时间（月）',
#               '账期订单占比','交易额总量','交易总量','交易量最大值','11天以上占比','最近连续交易月数','账期合作时间（月）',
#               '采购聚集程度（订单总数/供应商数）','订单总数','label']
# samples = samples[cols_filter]
# samples.shape

In [44]:
#分账期
# samples =pd.read_csv('./data/samples_7_fillna_nor.csv',index_col=0)
# samples =pd.read_csv('./data/samples_15_fillna_nor.csv')
# samples =pd.read_csv('./data/samples_30_fillna_nor.csv')
# cols_filter =['实收资本','注册时间','流动比率','资产负债率','存货周转天数','净资产','主营业务利润率','应收账款周转天数',
#               '下游客户情况','征信不良', '征信关注', '征信欠息', '征信正常','涉诉信息','有无贷款','提前还款占比','逾期占比',
#               '5-10天占比','账期订单交易量均值',
#               '交易量方差','交易量最小值','4天以内占比','交易额最小值','交易量均值','采购频率（订单总数/合作月数）','供应商数量',
#               '交易额均值','账期订单交易额均值','交易额方差','交易稳定性（近三个月交易量变化均值）','交易额最大值','合作时间（月）',
#               '账期订单占比','交易额总量','交易总量','交易量最大值','11天以上占比','最近连续交易月数','账期合作时间（月）',
#               '采购聚集程度（订单总数/供应商数）','订单总数','label']
# samples = samples[cols_filter]
# samples.shape

In [45]:
#变量筛选分段测试
# samples =pd.read_csv('./data/samples_sub_fillna_nor.csv',index_col=0)
# samples.head(),samples.shape

In [46]:
#以客户为样本
# samples =pd.read_csv('./data/samples_kehu_all_fillna_nor.csv',index_col=0)
# cols_filter =['实收资本','注册时间','流动比率','资产负债率','存货周转天数','净资产','主营业务利润率','应收账款周转天数',
#               '下游客户情况','企业征信','涉诉信息','有无贷款','账期订单交易量均值','交易量方差','交易量最小值','交易额最小值',
#               '交易量均值','采购频率（订单总数/合作月数）','供应商数量',
#               '交易额均值','账期订单交易额均值','交易额方差','交易稳定性（近三个月交易量变化均值）','交易额最大值','合作时间（月）',
#               '账期订单占比','交易额总量','交易总量','交易量最大值','最近连续交易月数','账期合作时间（月）',
#               '采购聚集程度（订单总数/供应商数）','订单总数','label']
# samples = samples[cols_filter]
# samples.shape

In [47]:
#求ks值
def ks_value(y_true,y_pre):
    fpr,tpr,t = roc_curve(y_true,y_pre)
    gap =tpr-fpr
    ks_value = np.max(gap)
    auc_value = auc(fpr,tpr)
    return ks_value,auc_value
    
def ks_plot(y_true,y_pre,model_name=None):
    plt.figure()
    fpr,tpr,t = roc_curve(y_true,y_pre)
    plt.plot(fpr,label='False Positive Rate')
    plt.plot(tpr,label='Ture Positive Rate')

    gap = tpr-fpr
    ks_value = np.max(gap)
    ks_index = np.where(gap == ks_value)
    y = [fpr[ks_index], tpr[ks_index]]
    x = [int(ks_index[0]), int(ks_index[0])]
    
    plt.plot(x,y,label='KS_max=%.2f'%ks_value)

    plt.title("%s KS graph\n th=%0.3f" % (model_name,t[ks_index]))
    plt.legend(loc="mid right")
    if model_name:
        plt.savefig('./new_figure/%s_%.2f.png'%(model_name,ks_value))
    plt.show()
    
def auc_plot(y_true,y_pre,model_name=None):
    plt.figure()
    fpr,tpr,th = roc_curve(y_true, y_pre)
    auc_value = auc(fpr,tpr)
    plt.plot(fpr,tpr,lw=1,label="ROC fold (AUC=%.2f)" % (auc_value))
    plt.xlim([-0.05,1.05])
    plt.ylim([0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("%s ROC curve"%model_name)
    plt.legend(loc='lower right')
    if model_name:
        plt.savefig('./new_figure/%s_%.2f.png'%(model_name,auc_value))
    plt.show()

In [48]:
#GBDT
def GBDT_CV():
    ks_list = []
    for train,test in skf:
        train_data = samples.iloc[train]
        test_data = samples.iloc[test]
        test_ma=xgb.DMatrix(test_data.iloc[:,:-1],test_data.iloc[:,-1])
        gbdt,auc = model_xgb(train_data,test_data)
        y_pro = gbdt.predict(test_ma)
        ks = ks_value(test_data.iloc[:,-1],y_pro)
        ks_list.append(ks)
    print ks_list
    return np.mean([x[0] for x in ks_list])

In [49]:
def model_CV(model_name):
    ks_list = []
#     weight = []
#     bias =[]
    for train,test in skf:
        train_data = samples.iloc[train]
        test_data = samples.iloc[test]
        model = model_name(train_data.iloc[:,:-1],train_data.iloc[:,-1])
        y_pro = model.predict_proba(test_data.iloc[:,:-1])
        y_pro = [x[1] for x in y_pro]
        ks = ks_value(test_data.iloc[:,-1],y_pro)        
        ks_list.append(ks)
#         weight.append(list(model.coef_[0]))
#         bias.append(list(model.intercept_))
    print ks_list
    return np.mean([x[0] for x in ks_list])

In [54]:
#季度统计订单样本
# samples = pd.read_csv('./new_out/3m_samples_all_fillna_nor.csv')
# samples =pd.read_csv('./new_out/3m_samples_all_add_fillna_nor.csv')
# samples =pd.read_csv('./new_out/3m_samples_woe_fillna_nor.csv') #所有变量，离散变量先缺失值填充0，再woe编码
# samples =pd.read_csv('./new_out/3m_samples_woe.csv') #所有变量woe编码
# samples = pd.read_csv('./new_out/old_custom_samples_3m_fillna_nor.csv')#老客户
# samples = pd.read_csv('./out1/samples_41_fillna_mean.csv',index_col=0) #离散变量woe编码，连续变量归一化，所有变量缺失值填均值
samples = pd.read_csv('./out1/samples_for_card_fillna_mean.csv',index_col=0)
skf = StratifiedKFold(samples.iloc[:,-1],n_folds=5)
samples.shape

(1391, 46)

In [10]:
skf

sklearn.cross_validation.StratifiedKFold(labels=[1 0 0 ..., 0 0 0], n_folds=5, shuffle=False, random_state=None)

In [128]:
lr_ks = model_CV(model_lr)
lr_ks

[(0.35531775496833373, 0.61356191308145891), (0.60098687899517778, 0.87226645732869801), (0.29595155321296401, 0.65633060446338454), (0.29213861164068633, 0.61775260737916349), (0.48715935852865316, 0.79735336996747785)]


0.40631083146916297

In [126]:
nn =model_CV(model_NN)
print nn

[(0.39266215330858267, 0.63452718934265118), (0.59952898957048339, 0.86419199282269821), (0.35022989794774034, 0.64657395985196808), (0.3689581697880453, 0.66081641807782887), (0.474486935067848, 0.77447572053381186)]
0.437173229137


In [134]:
print np.mean(np.mat(weight), axis=0)
tmp = pd.DataFrame(weight).T
tmp ['weight_avg'] = np.mean(np.mat(weight), axis=0).T
tmp['col'] = list(samples.iloc[:,:-1].columns)
tmp

[]


ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series

In [51]:
svm_ks =model_CV(model_svm)
svm_ks

[(0.39801266652107453, 0.72532212273422147), (0.4872715038690143, 0.76180329707300665), (0.3605472692609622, 0.68649770102052265), (0.47493551642929244, 0.75714926544802064), (0.43927329819445998, 0.75485028597061798)]


0.43200805085496069

In [ ]:
nb_ks = model_CV(model_NB)
nb_ks

In [ ]:
knn = model_CV(model_KNN)
knn

In [ ]:
dt =model_CV(model_DT)
dt

In [55]:
gbdt_ks =GBDT_CV()
gbdt_ks

[(0.42683992138021398, 0.71331076654291325), (0.2715038690142425, 0.6613210721094539), (0.32533363238757435, 0.66485365033082866), (0.4956824043960974, 0.77800829875518673), (0.3396882359537961, 0.69373107547381407)]


0.3718096126263849

In [ ]:
# from sklearn import cross_validation
# cv = StratifiedKFold(samples.iloc[:,-1],n_folds=5)
# lg = LogisticRegressionCV(class_weight='balanced')
# cv_lr = cross_validation.cross_val_score(lg,samples.iloc[:,:-1],samples.iloc[:,-1],cv=cv)
# cv_lr

### 最佳训练数据集

In [ ]:
count=0
for train,test in skf:
    count +=1
    train_data = samples.iloc[train]
    test_data = samples.iloc[test]
    train_data.to_csv('./data/model_train_data_%d.csv'%count)
    test_data.to_csv('./data/model_test_data_%d.csv'%count)
train_data = pd.read_csv('./data/model_train_data_5.csv',index_col=0)
test_data = pd.read_csv('./data/model_test_data_5.csv',index_col=0)

In [ ]:
lr = model_lr(train_data.iloc[:,:-1],train_data.iloc[:,-1])
y_pro = lr.predict_proba(test_data.iloc[:,:-1])
y_pro = [x[1] for x in y_pro]
ks_plot(test_data.iloc[:,-1],y_pro,'LR_ks_3m_sub_woe')
auc_plot(test_data.iloc[:,-1],y_pro,'LR_auc_3m_sub_woe')

In [ ]:
m_svm = model_svm(train_data.iloc[:,:-1],train_data.iloc[:,-1])
y_pro = m_svm.predict_proba(test_data.iloc[:,:-1])
y_pro = [x[1] for x in y_pro]
ks_plot(test_data.iloc[:,-1],y_pro,'svm_ks_3m_sub_woe')
auc_plot(test_data.iloc[:,-1],y_pro,'svm_auc_3m_sub_woe')

In [ ]:
nn = MLPClassifier(activation='logistic')
nn.fit(train_data.iloc[:,:-1],train_data.iloc[:,-1])


In [ ]:
nn.n_layers_